## The first step is setting up the IBC light clients that live on each counterpart's chain. We will heavily utilize terra.py (https://github.com/terra-money/terra.py) and terra_proto (https://github.com/terra-money/terra.proto) in this series, but you can absolutely use more generalized packages to interact with lcd endpoints and generate proto classes.

## Additionally, we use Setten (https://setten.io/) for private terra rpc endpoints; Osmosis provides a public rpc that we can use.

In [1]:
#imports

import pandas as pd
import json
import os
import sys
import base64
import requests
import subprocess
import math
import hashlib
import bech32
import time

from dateutil.parser import parse
from datetime import datetime, timedelta
from ecdsa import SECP256k1, SigningKey
from ecdsa.util import sigencode_string_canonize
from bech32 import bech32_decode, bech32_encode, convertbits

from terra_sdk.client.lcd import LCDClient
from terra_sdk.core.wasm import MsgStoreCode, MsgInstantiateContract, MsgExecuteContract
from terra_sdk.core.bank import MsgSend
from terra_sdk.core.fee import Fee
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.core.bech32 import get_bech
from terra_sdk.core import AccAddress, Coin, Coins
from terra_sdk.client.lcd.api.tx import CreateTxOptions, SignerOptions
from terra_sdk.client.localterra import LocalTerra
from terra_sdk.core.wasm.data import AccessConfig
from terra_sdk.client.lcd.api._base import BaseAsyncAPI, sync_bind

from terra_proto.cosmwasm.wasm.v1 import AccessType
from terra_proto.cosmos.tx.v1beta1 import Tx, TxBody, AuthInfo, SignDoc, SignerInfo, ModeInfo, ModeInfoSingle, BroadcastTxResponse
from terra_proto.cosmos.base.abci.v1beta1 import TxResponse
from terra_proto.cosmos.tx.signing.v1beta1 import SignMode
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse
from terra_proto.ibc.core.channel.v1 import MsgChannelOpenInit, Channel, State, Order, Counterparty, MsgChannelOpenTry, MsgChannelOpenAck, MsgChannelOpenConfirm, QueryUnreceivedPacketsRequest, QueryUnreceivedPacketsResponse, QueryPacketCommitmentRequest, QueryPacketCommitmentResponse, Packet, QueryNextSequenceReceiveRequest, QueryNextSequenceReceiveResponse, MsgRecvPacket, MsgTimeout, QueryUnreceivedAcksRequest, QueryUnreceivedAcksResponse, MsgAcknowledgement
from terra_proto.ibc.core.connection.v1 import MsgConnectionOpenInit, Counterparty as ConnectionCounterParty, Version, MsgConnectionOpenTry, MsgConnectionOpenAck, MsgConnectionOpenConfirm
from terra_proto.ibc.lightclients.tendermint.v1 import ClientState, ConsensusState, Fraction, Header
from terra_proto.ics23 import HashOp, LengthOp, LeafOp, InnerOp, ProofSpec, InnerSpec, CommitmentProof, ExistenceProof, NonExistenceProof, BatchProof, CompressedBatchProof, BatchEntry, CompressedBatchEntry, CompressedExistenceProof, CompressedNonExistenceProof
from terra_proto.ibc.core.commitment.v1 import MerkleRoot, MerklePrefix, MerkleProof
from terra_proto.tendermint.types import ValidatorSet, Validator, SignedHeader, Header as tendermintHeader, Commit, BlockId, PartSetHeader, CommitSig, BlockIdFlag
from terra_proto.tendermint.version import Consensus
from terra_proto.tendermint.crypto import PublicKey
from betterproto.lib.google.protobuf import Any
from betterproto import Timestamp

In [2]:
#misc helper functions
sys.path.append(os.path.join(os.path.dirname(__name__), '..', 'scripts'))

from helpers import proto_to_binary, timestamp_string_to_proto, stargate_msg, create_ibc_client, fetch_chain_objects

## We can actually use terra's lcd client & wallet classes for pretty much any cosmos chain.

In [3]:
#(terra, wallet, terra_rpc_url, terra_rpc_header) = fetch_chain_objects("localterra")
(inj, inj_wallet, inj_rpc_url, inj_rpc_header) = fetch_chain_objects("injective-888")
(osmo, osmo_wallet, osmo_rpc_url, osmo_rpc_header) = fetch_chain_objects("osmo-test-4")

## Terra provides the python protobuf classes (via betterproto) for ibc/tendermint; again other python-ized protos may be used (or self-generated)

In [4]:
#terra_proto imports
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse
from terra_proto.ibc.lightclients.tendermint.v1 import ClientState, ConsensusState, Fraction, Header
from terra_proto.ics23 import HashOp, LengthOp, LeafOp, InnerOp, ProofSpec, InnerSpec, CommitmentProof, ExistenceProof, NonExistenceProof, BatchProof, CompressedBatchProof, BatchEntry, CompressedBatchEntry, CompressedExistenceProof, CompressedNonExistenceProof
from terra_proto.ibc.core.commitment.v1 import MerkleRoot, MerklePrefix, MerkleProof
from terra_proto.tendermint.version import Consensus


## Create ibc light clients by submitting stargate messages containing context data from the counterpart chain

In [5]:
#fetch osmosis tendermint data
unbonding_period = int(osmo.staking.parameters()["unbonding_time"].replace('s', ''))
trusting_period = math.floor(unbonding_period * 2 / 3)
max_clock_drift = 20
latest_height_revision_number = 4
tendermint_info = osmo.tendermint.block_info()["block"]

print(f"""
osmosis information for client on injective

unbonding_period: {unbonding_period}
trusting_period: {trusting_period}
max_clock_drift: {max_clock_drift}
tendermint_info: {tendermint_info}
""")

#create the ibc client on injective, containing fetched osmosis tendermint data
msg = MsgCreateClient(
client_state=Any(
  type_url="/ibc.lightclients.tendermint.v1.ClientState",
  value=ClientState(
    chain_id=osmo.chain_id,
    trust_level=Fraction(1,3),
    trusting_period=timedelta(seconds=trusting_period),
    unbonding_period=timedelta(seconds=unbonding_period),
    max_clock_drift=timedelta(seconds=max_clock_drift),
    frozen_height=Height(0,0),
    latest_height=Height(latest_height_revision_number, int(tendermint_info["header"]["height"])),
    proof_specs=[
      ProofSpec(
        leaf_spec=LeafOp(
          hash=HashOp.SHA256,
          prehash_key=HashOp.NO_HASH,
          prehash_value=HashOp.SHA256,
          length=LengthOp.VAR_PROTO,
          prefix=base64.b64decode(b"AA=="),
        ),
        inner_spec=InnerSpec(
          child_order=[0,1],
          child_size=33,
          min_prefix_length=4,
          max_prefix_length=12,
          #empty_child=b'',
          hash=HashOp.SHA256,
        ),
        max_depth=0,
        min_depth=0
      ),
      ProofSpec(
        leaf_spec=LeafOp(
          hash=HashOp.SHA256,
          prehash_key=HashOp.NO_HASH,
          prehash_value=HashOp.SHA256,
          length=LengthOp.VAR_PROTO,
          prefix=base64.b64decode(b"AA=="),
        ),
        inner_spec=InnerSpec(
          child_order=[0,1],
          child_size=32,
          min_prefix_length=1,
          max_prefix_length=1,
          #empty_child=b'',
          hash=HashOp.SHA256,
        ),
        max_depth=0,
        min_depth=0
      ),
    ],
    upgrade_path=["upgrade", "upgradedIBCState"],
    allow_update_after_expiry=True,
    allow_update_after_misbehaviour=True,
  ).SerializeToString()
),
consensus_state=Any(
  type_url="/ibc.lightclients.tendermint.v1.ConsensusState",
  value=ConsensusState(
    timestamp=timestamp_string_to_proto(tendermint_info["header"]["time"]),
    root=MerkleRoot(base64.b64decode(tendermint_info["header"]["app_hash"])),
    next_validators_hash=base64.b64decode(tendermint_info["header"]["next_validators_hash"]),
  ).SerializeToString(),
),
signer=inj_wallet.key.acc_address,
)

print(f"""

ibc client creation message to be run on inj

{msg.to_dict()}

""")

#dispatch the message
result = stargate_msg("/ibc.core.client.v1.MsgCreateClient", msg, inj_wallet, inj)
result_df = pd.DataFrame(result["tx_response"]["logs"][0]["events"][0]["attributes"])
client_id_on_inj = result_df[result_df["key"]=="client_id"]["value"].values[0]

print(f"""

client_id on injective: {client_id_on_inj}

""")


osmosis information for client on injective

unbonding_period: 1209600
trusting_period: 806400
max_clock_drift: 20
tendermint_info: {'header': {'version': {'block': '11', 'app': '16'}, 'chain_id': 'osmo-test-4', 'height': '9559370', 'time': '2023-03-14T20:18:14.485562343Z', 'last_block_id': {'hash': '0pHsWrAD3nUwIBcaIogSlMQxhc59gCJKtFugMdEcAVg=', 'part_set_header': {'total': 1, 'hash': 'tKp8TzeqGxb5g250omg96z3GVxYaWaADq1zIN4zgmhM='}}, 'last_commit_hash': 'fSIafhIwwGf8vXuLfMKgj9aXAvkCS5bMNXtsq1vEXsQ=', 'data_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=', 'validators_hash': 'OKO5EbUTO5Td7kbtkXO7P2s/nycejIJi2GP2D/Cz8ro=', 'next_validators_hash': 'OKO5EbUTO5Td7kbtkXO7P2s/nycejIJi2GP2D/Cz8ro=', 'consensus_hash': 'qWfVX6y7oZq5YUkEjyR2xGV+wD0lt4qBr1uPCgj2Hf8=', 'app_hash': 'uwLMgyTiSwJr10Hw03VXlVYTv7u87YttxcCuefAoQ5o=', 'last_results_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=', 'evidence_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=', 'proposer_address': 'VpZxag4yMaD

In [19]:
#use helper function, create_ibc_client, to fetch injective tendermint data, fabricate the MsgCreateClient msg, and dispatch to osmosis chain
#ie. execute same steps as previous cell, but with reversed roles (injective consensus info on osmo chain)

result = create_ibc_client(inj, osmo, osmo_wallet, 888)
result_df = pd.DataFrame(result["tx_response"]["logs"][0]["events"][0]["attributes"])
client_id_on_osmo = result_df[result_df["key"]=="client_id"]["value"].values[0]

print(f"""

client_id on osmo: {client_id_on_osmo}

""")


  injective-888 information for client on osmo-test-4 

  unbonding_period: 1814400
  trusting_period: 1209600
  max_clock_drift: 20
  tendermint_info: {'header': {'version': {'block': '11', 'app': '0'}, 'chain_id': 'injective-888', 'height': '9199589', 'time': '2023-03-14T20:24:26.883970300Z', 'last_block_id': {'hash': '16CQZ9V6bJj6M1K30Fth//FWVKTTFaLR4uvBioyhYFg=', 'part_set_header': {'total': 1, 'hash': 'W8u7T9Gx7KB1KwiKGWG8JtIEWWhXamldx4leBnAEznc='}}, 'last_commit_hash': 'jIiBA32+36vtpmfaZ0u8dbCohUrFZqFFvm7GEI2Kxx4=', 'data_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=', 'validators_hash': 'NZMqGpR9oVa/krke1jgDJlhF56y9IG0E0xOYrBWUJ6w=', 'next_validators_hash': 'NZMqGpR9oVa/krke1jgDJlhF56y9IG0E0xOYrBWUJ6w=', 'consensus_hash': 'vml94SL+cPwUT5B4L5ZAe/3n0gSLsv2tqMpUVEVA8gg=', 'app_hash': 'vfkLQmiu4wFT3xoDoflr/4v1w0UCWWDbvsB9SC12/5Q=', 'last_results_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=', 'evidence_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=', 'proposer_a

## The key results from this step are the client_id's built each on injective and osmosis. Let's take a look at the client_id's, and the underlying client state data stored onchain.

In [20]:
#client_id on injective, and its underlying state representing a snapshot summary of osmosis's consensus/tendermint state
client_state_on_inj = inj.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_inj}")

print(f"""

client_id on inj: {client_id_on_inj}
client_state on inj: {client_state_on_inj}

""")



client_id on inj: 07-tendermint-87
client_state on inj: {'client_state': {'@type': '/ibc.lightclients.tendermint.v1.ClientState', 'chain_id': 'osmo-test-4', 'trust_level': {'numerator': '1', 'denominator': '3'}, 'trusting_period': '806400s', 'unbonding_period': '1209600s', 'max_clock_drift': '20s', 'frozen_height': {'revision_number': '0', 'revision_height': '0'}, 'latest_height': {'revision_number': '4', 'revision_height': '9559370'}, 'proof_specs': [{'leaf_spec': {'hash': 'SHA256', 'prehash_key': 'NO_HASH', 'prehash_value': 'SHA256', 'length': 'VAR_PROTO', 'prefix': 'AA=='}, 'inner_spec': {'child_order': [0, 1], 'child_size': 33, 'min_prefix_length': 4, 'max_prefix_length': 12, 'empty_child': None, 'hash': 'SHA256'}, 'max_depth': 0, 'min_depth': 0}, {'leaf_spec': {'hash': 'SHA256', 'prehash_key': 'NO_HASH', 'prehash_value': 'SHA256', 'length': 'VAR_PROTO', 'prefix': 'AA=='}, 'inner_spec': {'child_order': [0, 1], 'child_size': 32, 'min_prefix_length': 1, 'max_prefix_length': 1, 'emp

In [21]:
#client_id on osmosis, and its underlying state representing a snapshot summary of terra's consensus/tendermint state
client_state_on_osmo = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")

print(f"""

client_id on osmosis: {client_id_on_osmo}
client_state on osmosis: {client_state_on_osmo}

""")



client_id on osmosis: 07-tendermint-4298
client_state on osmosis: {'client_state': {'@type': '/ibc.lightclients.tendermint.v1.ClientState', 'chain_id': 'injective-888', 'trust_level': {'numerator': '1', 'denominator': '3'}, 'trusting_period': '1209600s', 'unbonding_period': '1814400s', 'max_clock_drift': '20s', 'frozen_height': {'revision_number': '0', 'revision_height': '0'}, 'latest_height': {'revision_number': '888', 'revision_height': '9199589'}, 'proof_specs': [{'leaf_spec': {'hash': 'SHA256', 'prehash_key': 'NO_HASH', 'prehash_value': 'SHA256', 'length': 'VAR_PROTO', 'prefix': 'AA=='}, 'inner_spec': {'child_order': [0, 1], 'child_size': 33, 'min_prefix_length': 4, 'max_prefix_length': 12, 'empty_child': None, 'hash': 'SHA256'}, 'max_depth': 0, 'min_depth': 0}, {'leaf_spec': {'hash': 'SHA256', 'prehash_key': 'NO_HASH', 'prehash_value': 'SHA256', 'length': 'VAR_PROTO', 'prefix': 'AA=='}, 'inner_spec': {'child_order': [0, 1], 'child_size': 32, 'min_prefix_length': 1, 'max_prefix_l

## Persist client_id's for next steps.

In [22]:
#save to current directory

data = {
    "client_id_on_inj": client_id_on_inj,
    "client_id_on_osmo": client_id_on_osmo,
}

with open("context.json", "w") as f:
    # Write the dictionary to the file as a JSON string
    json.dump(data, f)


## And that's it for IBC client setup! Please continue to the next notebook to create an ibc connection between terra and osmosis.